# Title: Accuracy of Predicting Newsletter Subscription by Looking at Player's Age and Hours Played #

## Introduction ##
##### Understanding user engagement in online platforms such as video games is critical for developers and researchers. In this project, I analyze data from a Minecraft research server to determine if using players' **age** and **hours played** can be a reliable way to predict whether they subscribe to a game-related newsletter.The aim of this project is to help a research group at UBC target their recruitment efforts #####

##### For this project I used the data players.csv, which contains the following variables: #####
| Variable       | Description                                      | Type        |
|----------------|--------------------------------------------------|-------------|
| experience     | Experience metric (not used in this analysis)    | Numeric     |
| subscribe      | Whether the player subscribed to the newsletter  | Logical |
| hashedEmail    | Unique identifier for player (not used)          | Text        |
| played_hours   | Total hours the player played                    | Numeric     |
| name           | Player name (not used)                           | Text        |
| gender         | Player gender (not used)                         | Text        |
| Age            | Player age                                       | Numeric     |


##### This dataset contains 197 observations. #####

## Methods And Results ##

In [ ]:
#Load Libraries
library(tidyverse)
library(repr)
library(tidymodels)

1. The first thing I will do is to explore the data so we can prepare it for our analysis. 

In [ ]:
# Explore the Data 
players<- read_csv("players.csv")

2. Now that I have explored the data, I could see what each variable is and what type of variable they are. Since I am trying to see if Age and Hours Played are good predictors of the player subscribing to the newsletter, I will `select` only those 3 variables and also `mutate` the *subscribe* variable into a factor instead of logical and drop the na in the dataset. 

In [ ]:
#Load and clean the data
players<- read_csv("players.csv") |> 
filter(!is.na(Age)) |>
select(subscribe, played_hours, Age)|> 
mutate(subscribe= as.factor(subscribe))  

players

3. Now that I have loaded, explored and cleaned the data, I will see what is the average age and hours played of the players.

In [ ]:
#Summarize the mean age and mean hours played so we can have an idea of the profile of an average player
players_summary <- players |>
summarize(mean_age = mean(Age, na.rm = TRUE),
mean_hours = mean(played_hours, na.rm = TRUE))
players_summary

### *Step 1*: Data Splitting
For us to find the best number of K as well as test if our classifier is a good model we will split the data into a training and test subset.

In [ ]:
set.seed(123)
players_split <- initial_split(players, prop = 0.7, strata = subscribe)
players_train <- training(players_split)
players_test <- testing(players_split)

### *Step 2*: Preprocessing Recipe
I set my recipe and standardized the predictors. 

In [ ]:
players_recipe <- recipe(subscribe ~ played_hours + Age, data = players_train) |>
step_center(all_predictors()) |>
step_scale(all_predictors())


### *Step 3*: Specify KNN Model 

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
set_engine("kknn") |>
set_mode("classification")